# Imports

In [1]:
# !pip install -Iv transformers==4.38.1
# !pip install -Iv datasets==2.1.0
# !pip install -Iv tensorflow==2.15.0
# !pip install -Iv keras==3.0.5
# !pip install tf-keras
# !pip install -Iv nltk==3.2.4
# !pip install -Iv contractions==0.1.73
# !pip install -Iv accelerate==0.27.2

In [2]:
import pandas as pd
import os 
import sklearn
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset, load_metric
import datasets
import torch
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding, DistilBertConfig, DistilBertForSequenceClassification, BertConfig, BertForSequenceClassification, create_optimizer, TrainingArguments, Trainer
from transformers.keras_callbacks import KerasMetricCallback
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import pipeline
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-13 05:44:21.397020: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 05:44:22.196517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/h

# Import Data

In [3]:
train_data_path = 'data/train.csv'
test_data_path = 'data/test.csv'

In [4]:
labelled_data = pd.read_csv(train_data_path)
labelled_data.head(5)

,text,stars
0,WILL NEVER COME BACK! HORRIBLE SERVICE & NASTY...,1
1,Terrible food. Terrible service.\n\nThe absol...,1
2,"So, right away if I go into a buffet setting, ...",2
3,I have gotten good cuts from this place. I eve...,3
4,I felt this place was a bit lackluster conside...,2


In [5]:
labelled_data['stars'] = labelled_data['stars'].replace({1: 'one_star', 2: 'two_stars', 3: 'three_stars'})

In [6]:
labelled_data.head(5)

,text,stars
0,WILL NEVER COME BACK! HORRIBLE SERVICE & NASTY...,one_star
1,Terrible food. Terrible service.\n\nThe absol...,one_star
2,"So, right away if I go into a buffet setting, ...",two_stars
3,I have gotten good cuts from this place. I eve...,three_stars
4,I felt this place was a bit lackluster conside...,two_stars


# Preprocessing

In [7]:
# one hot encode the labels 
one_hot_labels = pd.get_dummies(labelled_data['stars'])
# # drop the old 'stars' column to replace with the one hot encoded values 
# labelled_data = labelled_data.drop('stars', axis= 1)
# join with the one hot encoded 
labelled_data = labelled_data.join(one_hot_labels)

labelled_data.head(10)

,text,stars,one_star,three_stars,two_stars
0,WILL NEVER COME BACK! HORRIBLE SERVICE & NASTY...,one_star,True,False,False
1,Terrible food. Terrible service.\n\nThe absol...,one_star,True,False,False
2,"So, right away if I go into a buffet setting, ...",two_stars,False,False,True
3,I have gotten good cuts from this place. I eve...,three_stars,False,True,False
4,I felt this place was a bit lackluster conside...,two_stars,False,False,True
5,Back when I lived in Pollock Halls in my first...,three_stars,False,True,False
6,Short version:\n- 20 stars for having free vod...,two_stars,False,False,True
7,Okay.....read the other reviews about taking t...,one_star,True,False,False
8,"The food is really good, but we had one big co...",three_stars,False,True,False
9,"Cool place, comfortable. Sorely needed in the ...",three_stars,False,True,False


In [8]:
# # 1. Expand Contractions using contractions library
# def expand_contractions(text):
#     return contractions.fix(text)

# # Apply expand_contractions function to 'text' column
# labelled_data['text'] = labelled_data['text'].apply(expand_contractions)

# # 2. Lower Case
# labelled_data['text'] = labelled_data['text'].str.lower()

# # 3. Remove Punctuations
# labelled_data['text'] = labelled_data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# # 4. Remove Extra Spaces
# labelled_data['text'] = labelled_data['text'].apply(lambda x: re.sub(' +', ' ', x.strip()))

In [9]:
# # Download the WordNet resource
# nltk.download('stopwords')

In [10]:
# # 5. Remove Stopwords
# stop_words = set(stopwords.words('english'))
# labelled_data['text'] = labelled_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

In [11]:
# # Download the WordNet resource
# nltk.download('wordnet')

In [12]:
# !unzip /home/ec2-user/nltk_data/corpora/wordnet.zip -d /home/ec2-user/nltk_data/corpora/

In [13]:
# # 6. Stemming and Lemmatization
# lemmatizer = WordNetLemmatizer()
# def lemmatize_words(text):
#     return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
# labelled_data['text'] = labelled_data['text'].apply(lemmatize_words)

In [14]:
# labelled_data.head(10)

# Split Data

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(
    labelled_data['text'], 
    labelled_data[['one_star', 'two_stars', 'three_stars']], 
    test_size= 0.2, 
    random_state= 42, 
    stratify= labelled_data['stars']
)

In [16]:
print(y_train.value_counts())
print(y_valid.value_counts())

one_star  two_stars  three_stars
False     False      True           1600
          True       False          1600
True      False      False          1600
Name: count, dtype: int64
one_star  two_stars  three_stars
False     False      True           400
          True       False          400
True      False      False          400
Name: count, dtype: int64


In [17]:
# X_train

In [18]:
# y_train

In [19]:
train_ind = X_train.index.values.tolist()
valid_ind = X_valid.index.values.tolist()

train_data = {'text': [], 'one_star': [], 'two_stars': [], 'three_stars': []}
valid_data = {'text': [], 'one_star': [], 'two_stars': [], 'three_stars': []}

for ti in train_ind:
    train_data['text'].append(X_train[ti])
    train_data['one_star'].append(y_train.loc[ti]['one_star'])
    train_data['two_stars'].append(y_train.loc[ti]['two_stars'])
    train_data['three_stars'].append(y_train.loc[ti]['three_stars'])

for vi in valid_ind:
    valid_data['text'].append(X_valid[vi])
    valid_data['one_star'].append(y_valid.loc[vi]['one_star'])
    valid_data['two_stars'].append(y_valid.loc[vi]['two_stars'])
    valid_data['three_stars'].append(y_valid.loc[vi]['three_stars'])
    
print(len(train_data['text']))
print(len(valid_data['text']))

4800
1200


In [20]:
train_dataset = Dataset.from_dict(train_data)
valid_dataset = Dataset.from_dict(valid_data)
data = datasets.DatasetDict({"train": train_dataset,"valid": valid_dataset})

In [21]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'one_star', 'two_stars', 'three_stars'],
        num_rows: 4800
    })
    valid: Dataset({
        features: ['text', 'one_star', 'two_stars', 'three_stars'],
        num_rows: 1200
    })
})

In [22]:
data['train'][0]

{'text': "Remeber ten years ago, when Wired was publishing articles about how the Internet would end retail shopping?  Well, they were wrong.  Bad customer service will end retail shopping and this Best Buy is a fine example of the downward trend.  The people that work at this ubiquitous big box store are assholes that usually don't know anything about their products and could give two fucks about whether or not you're being helped.    \\r\\n\\r\\nThat said, man, do I love gadgets.  And Best Buy's got a lot of 'em.",
 'one_star': False,
 'two_stars': True,
 'three_stars': False}

In [23]:
labels = [label for label in data['train'].features.keys() if label not in ['text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['one_star', 'two_stars', 'three_stars']

In [24]:
id2label

{0: 'one_star', 1: 'two_stars', 2: 'three_stars'}

# Tokenizer

In [25]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [26]:
tokenized_data = data.map(preprocess_data, batched=True, remove_columns= data['train'].column_names)

Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [27]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4800
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1200
    })
})

In [28]:
tokenized_data['train'][0]['labels']

[0.0, 1.0, 0.0]

In [29]:
tokenized_data.set_format("torch")

# Data Collator

In [30]:
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Model Building

In [31]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train Model Pt.1

In [32]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [33]:
batch_size = 8
metric_name = "f1"

In [34]:
args = TrainingArguments(
    f"bert-model-stat940",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [35]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

# Verify a batch and forward pass

In [36]:
tokenized_data['train'][0]['labels'].type()

'torch.FloatTensor'

In [37]:
tokenized_data['train']['input_ids'][0]

tensor([  101,  2128,  4168,  5677,  2702,  2086,  3283,  1010,  2043, 17502,
         2001,  4640,  4790,  2055,  2129,  1996,  4274,  2052,  2203,  7027,
         6023,  1029,  2092,  1010,  2027,  2020,  3308,  1012,  2919,  8013,
         2326,  2097,  2203,  7027,  6023,  1998,  2023,  2190,  4965,  2003,
         1037,  2986,  2742,  1997,  1996, 14047,  9874,  1012,  1996,  2111,
         2008,  2147,  2012,  2023, 28498,  2502,  3482,  3573,  2024, 22052,
         2015,  2008,  2788,  2123,  1005,  1056,  2113,  2505,  2055,  2037,
         3688,  1998,  2071,  2507,  2048,  6616,  2015,  2055,  3251,  2030,
         2025,  2017,  1005,  2128,  2108,  3271,  1012,  1032,  1054,  1032,
         1050,  1032,  1054,  1032, 23961, 12707,  2056,  1010,  2158,  1010,
         2079,  1045,  2293, 11721, 28682,  1012,  1998,  2190,  4965,  1005,
         1055,  2288,  1037,  2843,  1997,  1005,  7861,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [39]:
#forward pass
outputs = model(input_ids=tokenized_data['train']['input_ids'][0].unsqueeze(0), labels=tokenized_data['train'][0]['labels'].unsqueeze(0))
outputs

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


SequenceClassifierOutput(loss=tensor(0.6281, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0678,  0.0447, -0.3025]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# # Train Model Pt.2

In [40]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.534000,0.470162,0.615526,0.715208,0.558333
2,0.418500,0.468918,0.632933,0.725417,0.614167
3,0.332700,0.593690,0.646051,0.734583,0.640833
4,0.240800,0.756868,0.639933,0.730000,0.635000


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

# Save Model

In [ ]:
model.save_pretrained('models')

In [57]:
trainer.evaluate()

{'eval_loss': 0.7568681240081787,
 'eval_f1': 0.6399331662489557,
 'eval_roc_auc': 0.7299999999999999,
 'eval_accuracy': 0.635}

# Inference

In [42]:
test_data = pd.read_csv(test_data_path)
test_data.head(5)

,ID,text
0,1,\n\nDuring my recent company trip to our Tempe...
1,2,\n\nAfter hearing about Pauly D from Jersey Sh...
2,3,\n\nI had high hopes for this restaurant based...
3,4,We experienced overpriced and underwhelming en...
4,5,\n\nThe windows from this company are definite...


In [70]:
output_path = 'outputs/new_distil_bert_v02.csv'

In [71]:
count = 0
error_ind = []
with open(output_path, "w", newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["ID","Label"])
    for index, row in test_data.iterrows():
        text_id = row['ID']
        text = row['text']
        encoding = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
        # print(encoding)
        # print(encoding['attention_mask'].size())
        try:
            outputs = trainer.model(**encoding)
        except: 
            count += 1
            error_ind.append(text_id)
        logits = outputs.logits
        # apply sigmoid + threshold
        sigmoid = torch.nn.Sigmoid()
        probs = sigmoid(logits.squeeze().cpu())
        predictions = np.zeros(probs.shape)
        predictions[np.argmax(probs.detach().numpy())] = 1
        # turn predicted id's into actual label names
        predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
        
        if predicted_labels[0] == 'one_star':
            conv_pred = 1
        elif predicted_labels[0] == 'two_stars':
            conv_pred = 2
        else: 
            conv_pred = 3

        
        csv_writer.writerow([text_id,conv_pred])
    print(count)

0
